In [1]:
import lightning as L

from slrt.constants import *

/new_home/xzj23/.conda/envs/slrt/lib/python3.11/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
2024-12-04 21:47:58.713723: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-04 21:47:58.729817: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model_name = "MSKA"
dataset_name = "csl-daily-keypoint"
dataset_type = "keypoint"
tokenizer_name = "SimpleTokenizer"
ckpt_file = "/new_home/xzj23/workspace/SLRT/experiments/CSL-Daily_Experiment/MSKA/11/checkpoints/best.ckpt"
dataset_dir = "/new_home/xzj23/workspace/SLRT/data/csl-daily"
keypoints_file = "/new_home/xzj23/workspace/SLRT/data/preprocessed/csl-daily/csl-daily-keypoints-mska.pkl"
vocab_file = "/new_home/xzj23/workspace/SLRT/data/preprocessed/csl-daily/csl-daily-gloss-vocab.txt"

In [3]:
# load model
model = ModelClassDict[model_name].load_from_checkpoint(ckpt_file)

/new_home/xzj23/.conda/envs/slrt/lib/python3.11/site-packages/lightning/fabric/utilities/cloud_io.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, m

In [4]:
# get data
datamodule = DataModuleClassDict[dataset_name](
    keypoints_file=keypoints_file,
    batch_size=1, num_workers=10,
    transform=TransformDict[dataset_type],
    tokenizer=TokenizerDict[tokenizer_name](vocab_file=vocab_file),
)
datamodule.setup(stage="test")

dl = datamodule.test_dataloader()

In [5]:
# trainer
trainer = L.Trainer(accelerator='gpu', devices=[0], precision="16-mixed", logger=False)
res = trainer.predict(model=model, dataloaders=dl)

/new_home/xzj23/.conda/envs/slrt/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /new_home/xzj23/.conda/envs/slrt/lib/python3.11/site ...
  rank_zero_warn(
Using 16bit Automatic Mixed Precision (AMP)
/new_home/xzj23/.conda/envs/slrt/lib/python3.11/site-packages/lightning/pytorch/plugins/precision/amp.py:53: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: ModelCheckpoin

Predicting: 0it [00:00, ?it/s]

In [6]:
for item in res:
    print(f"{item[0][0]:<50}: {' '.join(item[1][0])}")

S000020_P0004_T00                                 : 他 今天 年龄
S000040_P0008_T00                                 : 今天 星期
S000054_P0008_T00                                 : 今天 我 想 面条
S000153_P0000_T00                                 : 你 张 什么 时间 认识
S000185_P0004_T00                                 : 超市 我 要 买 椅子 你 去
S000185_P0008_T00                                 : 超市 我 要 买 椅子 你 来
S000195_P0004_T00                                 : 下午 你们 做 什么
S000196_P0008_T00                                 : 椅子 这 想 什么 时间 去 买
S000201_P0000_T00                                 : 我 每天 早 旅游 起床
S000213_P0004_T00                                 : 每天 来 累 多
S000213_P0008_T00                                 : 他 每天 衣服 来 累 多
S000218_P0004_T00                                 : 手表
S000218_P0008_T00                                 : 手表 手
S000220_P0000_T00                                 : 这 手表 钱 多少
S000224_P0000_T00                                 : 我 杯子 你 看
S000224_P0008_T00                                 : 我 杯子 你 看

In [11]:
# from slrt.models.utils.decoders.CTCBeamSearchDecoder import CTCBeamSearchDecoder
#
# trainer = L.Trainer(accelerator='gpu', devices=[0], precision="16-mixed",strategy="ddp_notebook", logger=False)
#
# ctc_decoder = CTCBeamSearchDecoder(
#     tokenizer=TokenizerDict[tokenizer_name](vocab_file=vocab_file),
#     beam_width=10,
#     num_processes=10
# )
#
# model.hparams.probs_decoder = ctc_decoder

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [12]:
# trainer.test(model=model, dataloaders=dl)

RuntimeError: Lightning can't create new processes if CUDA is already initialized. Did you manually call `torch.cuda.*` functions, have moved the model to the device, or allocated memory on the GPU any other way? Please remove any such calls, or change the selected strategy. You will have to restart the Python kernel.